In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy


directory = './ml-1m'
ratings = []
with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
    for l in f:
        user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
        ratings.append({
            'user_id': user_id,
            'movie_id': movie_id,
            'rating': rating,
            'timestamp': timestamp,
            })
ratings = pd.DataFrame(ratings)

ratings = ratings.drop(['timestamp'],axis=1)
print(ratings)
# 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
df = ratings.sample(frac=1).reset_index(drop=True)
filter_user = deepcopy(df)
counts = filter_user['user_id'].value_counts()
filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

filtered_df = deepcopy(filter_user)
counts = filtered_df['movie_id'].value_counts()
filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
filtered_df = filtered_df.reset_index(drop=True)
print(filtered_df)


neighbor_dict = {}
user = []
item = []

with open('real_train.txt', 'r') as f:
    for l in f.readlines():
        if len(l) > 0:
            l = l.strip('\n').split(' ')
            neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
            user.append(int(l[0]))

def jaccard_index(u_i, u_j, neighbor_dict):
    u_i_neighbor = neighbor_dict[u_i]
    u_j_neighbor = neighbor_dict[u_j]
    return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

def node_copying(u_j, u_m, user):
    if u_j in user and u_m in user:
        zeta_distribution = jaccard_index(u_j, u_m, neighbor_dict) / sum(jaccard_index(u_j, u_i, neighbor_dict) for u_i in user)
        return zeta_distribution
    else:
        return 0

print(node_copying(1,1, user))


class BGCF(nn.Module):
    pass

         user_id  movie_id  rating
0              1      1193       5
1              1       661       3
2              1       914       3
3              1      3408       4
4              1      2355       5
...          ...       ...     ...
1000204     6040      1091       1
1000205     6040      1094       5
1000206     6040       562       5
1000207     6040      1096       4
1000208     6040      1097       4

[1000209 rows x 3 columns]
        user_id  movie_id  rating
0           710       671       4
1          3323      2598       2
2           839       589       4
3          4897      2716       5
4          3762      2664       5
...         ...       ...     ...
998534     5182      1902       4
998535     4230      2096       4
998536      278      3837       4
998537     5570      3552       4
998538     4738       910       5

[998539 rows x 3 columns]
1.0
